In [ ]:
import os
from google.colab import userdata

api_key = userdata.get("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key
print("OK")


OK


In [ ]:
from google.colab import files
from google.colab import drive
from google.colab import userdata
import pandas as pd
import csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade langchain langchain_community

In [ ]:
!pip install -U langchain_openai

In [ ]:
output_file = "/content/drive/My Drive/active-bugs.csv"
gt_file = "/content/drive/My Drive/gt-summaries.csv"

df = pd.read_csv(output_file)


if 'example_summary ' in df.columns:
    df = df.drop(columns=['example_summary '])

gt = pd.read_csv(gt_file)
df = df.merge(gt, on='bug.id', how='left')

In [ ]:
filtered_df = df[
    (df['project.id'] == 3) &
    (df['report.url'].notna()) & (df['report.url'] != '') &
    (df['buggy.url'].notna()) & (df['buggy.url'] != '') &
    (df['fixed.url'].notna()) & (df['fixed.url'] != '') &
    (df['diff.url'].notna()) & (df['diff.url'] != '') &
    (df['bug_report'].notna()) & (df['bug_report'] != '')
]


In [ ]:
filtered_df.head()

,bug.id,project.name,project.id,revision.id.buggy,revision.id.fixed,report.id,report.url,buggy.url,fixed.url,diff.url,bug_report,buggy_code,patch_code,ground_truth_summary
65,66,Closure,3,2353d807058bc2a20af279a480d6652cdf892f4d,1dfad5043a207e032a78ef50c3cba50488bcd300,253,https://storage.googleapis.com/google-code-arc...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,Bug Report ID: 253\nStatus: Fixed\nSummary: fu...,/*\n * Copyright 2008 The Closure Compiler Aut...,Commit Message: fixed files form Closure#1\nFi...,Summary: function arguments should not be opti...
66,67,Closure,3,61095090415cff7cae4f3645fa76ee7cdd3ee23d,d1cfe67977d8f3aaa85ec20c262171da394d5977,884,https://storage.googleapis.com/google-code-arc...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,Bug Report ID: 884\nStatus: Fixed\nSummary: co...,/*\n * Copyright 2006 The Closure Compiler Aut...,Commit Message: fixed files form Closure#2\nFi...,combining @interface and multiple @extends can...
67,68,Closure,3,3cc85c3c37aa8bc834a4a86f91ddeb399d854024,d80fcc04239ab8c4cf781273c4f9bc54cf06f479,864,https://storage.googleapis.com/google-code-arc...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,Bug Report ID: 864\nStatus: Fixed\nSummary: op...,/*\n * Copyright 2009 The Closure Compiler Aut...,Commit Message: fixed files form Closure#3\nFi...,optimization fails with variable in catch clause
68,69,Closure,3,1c95684b4a6add525b3070cbd27c234981520676,efefb736fccc2039b5fb079710b3f2ac82b8c6e4,873,https://storage.googleapis.com/google-code-arc...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,Bug Report ID: 873\nStatus: Fixed\nSummary: Co...,/*\n *\n * ***** BEGIN LICENSE BLOCK *****\n *...,Commit Message: fixed files form Closure#4\nFi...,Converting from an interface type to a constru...
69,70,Closure,3,722d1192e7ed174a12911dce09594228e31240e9,59eec92e364b2ec2cec9dd63449f5c0134983f18,851,https://storage.googleapis.com/google-code-arc...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,Bug Report ID: 851\nStatus: Fixed\nSummary: Co...,/*\n * Copyright 2011 The Closure Compiler Aut...,Commit Message: fixed files form Closure#5\nFi...,"Compiler ignores 'delete' statements, can brea..."


In [ ]:
from langchain import PromptTemplate
from langchain_openai import OpenAI
from langchain.chains import LLMChain


In [ ]:
from langchain_openai import ChatOpenAI

llm = OpenAI(api_key=api_key, temperature=0.7)



In [ ]:
template = """
Given the bug report, Write a one-sentence summary of the core issue using no more than 10 words.\n

Bug Report:
{bug_report}

Summary:
"""

prompt = PromptTemplate(input_variables=["bug_report"], template=template)
chain = prompt | llm


In [ ]:
import time

In [ ]:
summaries = []

for index, row in filtered_df.iterrows():
    print(f"Processing index {index}...")

    bug_report = row['bug_report']
    bug_id = row['bug.id']

    try:
        result = chain.invoke({"bug_report": bug_report})
        print(result)
        summary = result.strip()
    except Exception as e:
        print(f"Error on index {index}: {e}")
        summary = "Summary not available"

    print(f"Generated summary: {summary}\n")
    summaries.append([bug_id, summary])

    time.sleep(0.1)


Processing index 65...
Function arguments are being optimized away, compromising the function's length property and causing issues with certain techniques like currying.
Generated summary: Function arguments are being optimized away, compromising the function's length property and causing issues with certain techniques like currying.

Processing index 66...

Compiler crashes when combining @interface and multiple @extends.
Generated summary: Compiler crashes when combining @interface and multiple @extends.

Processing index 67...
Optimization fails with variable in catch clause.
Generated summary: Optimization fails with variable in catch clause.

Processing index 68...
Stack overflow occurs when converting interface type to constructor.
Generated summary: Stack overflow occurs when converting interface type to constructor.

Processing index 69...
Compiler ignoring 'delete' statements causes functionality issues.
Generated summary: Compiler ignoring 'delete' statements causes functiona

In [ ]:
summaries

[[66,
  "Function arguments are being optimized away, compromising the function's length property and causing issues with certain techniques like currying."],
 [67, 'Compiler crashes when combining @interface and multiple @extends.'],
 [68, 'Optimization fails with variable in catch clause.'],
 [69, 'Stack overflow occurs when converting interface type to constructor.'],
 [70, "Compiler ignoring 'delete' statements causes functionality issues."],
 [71, "Inconsistent 'this' type checking causing duplicated bug reports."],
 [72, 'goog.isFunction and friends have bad type inference.'],
 [73,
  'Obfuscated code triggers TypeError in Firefox due to redeclaring variables.'],
 [74,
  'Compiler fails to find amd module in subdirectory, causing build error.'],
 [75,
  'Google Closure Compiler generates incorrect code when mixing types in ternary operator.'],
 [76, 'Invalid property not reported on function with @this annotation.'],
 [77, 'Try/catch blocks include unexpected code, causing incorr

In [ ]:
summary_df = pd.DataFrame(summaries, columns=['Bug ID', 'Summary'])
summary_df.to_csv("/content/drive/MyDrive/gpt3.5_bug_summaries_zero_shot.csv", index=False)


In [ ]:
llm = ChatOpenAI(api_key=api_key, temperature=0.7, model="gpt-3.5-turbo")

In [ ]:
example_bug_report = filtered_df.iloc[5]['bug_report']
example_summary = filtered_df.iloc[5]['ground_truth_summary']

In [ ]:
template = """Here is an example of a bug report and its summary:

Example Bug Report:
{example_bug_report}

Example Summary:
{example_summary}

Now, write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless it naturally applies; tailor the summary to the new bug report.

Bug Report:
{bug_report}

Summary:"""

prompt_template = PromptTemplate(
    input_variables=["example_bug_report", "example_summary", "bug_report"],
    template=template
)
chain = prompt_template | llm

In [ ]:
summaries = []

for index, row in filtered_df.iterrows():
    print(f"Processing index {index}...")

    bug_id = row['bug.id']
    bug_report = row['bug_report']

    try:
        response = chain.invoke({
            "example_bug_report": example_bug_report,
            "example_summary": example_summary,
            "bug_report": bug_report
        })
        print(response)
        summary_result = response.content.strip()
    except Exception as e:
        print(f"Error at index {index}: {e}")
        summary_result = "Summary not available"

    if not summary_result:
        summary_result = "Summary not available"

    print(f"Generated summary for index {index}: {summary_result}\n")
    summaries.append([bug_id, summary_result])
    time.sleep(0.1)

Processing index 65...
content='function arguments should not be optimized away' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 1505, 'total_tokens': 1512, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C2YumMABc6ieaN2RlOtD1exSnemwn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--79b0545b-e79c-4909-9e51-078ca678f981-0' usage_metadata={'input_tokens': 1505, 'output_tokens': 7, 'total_tokens': 1512, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Generated summary for index 65: function arguments should not be optimized away

Processing index 66...
content='Combining @interface and multiple @extend

In [ ]:
summary_df = pd.DataFrame(summaries, columns=['Bug ID', 'Summary'])
summary_df.to_csv("/content/drive/MyDrive/gpt3.5_bug_summaries_one_shot.csv", index=False)

In [ ]:
llm = ChatOpenAI(api_key=api_key, temperature=0.7, model="gpt-3.5-turbo")

In [ ]:
examples = []
example_indices = [11, 21, 31]

for i in example_indices:
    ex_bug = filtered_df.iloc[i]['bug_report']
    ex_sum = filtered_df.iloc[i]['ground_truth_summary']
    examples.append((ex_bug, ex_sum))

In [ ]:
few_shot_examples = ""
for i, (bug, summary) in enumerate(examples, 1):
    few_shot_examples += f"""Example {i} Bug Report:
{bug}

Example {i} Summary:
{summary}

"""
template = """
Here are a few examples of bug reports and their summaries:

{few_shot_examples}
Now, write a one-sentence summary of the core issue using no more than 10 words.
Tailor the summary to the new bug report. Avoid copying earlier examples unless they naturally apply.

Bug Report:
{bug_report}

Summary:
"""
prompt_template = PromptTemplate(
    input_variables=["few_shot_examples", "bug_report"],
    template=template
)

chain = prompt_template | llm

In [ ]:
template

'\nHere are a few examples of bug reports and their summaries:\n\n{few_shot_examples}\nNow, write a one-sentence summary of the core issue using no more than 10 words.\nTailor the summary to the new bug report. Avoid copying earlier examples unless they naturally apply.\n\nBug Report:\n{bug_report}\n\nSummary:\n'

In [ ]:
few_shot_examples

"Example 1 Bug Report:\nBug Report ID: 794\nStatus: Fixed\nSummary: Try/catch blocks incorporate code not inside original blocks\nLabels: Type-Defect, Priority-High\nStars: 0\nComment Count: 3\nComments:\n\n0. **Comment by User (ID: 8682039881586643116)**\n   - **Timestamp**: 1344189644\n   - **Content**: <b>What steps will reproduce the problem?</b>\n\nStarting with this code:\n\n-----\nfunction a() {\n var x = '1';\n try {\n  x += somefunction();\n } catch(e) {\n }\n x += &quot;2&quot;;\n try {\n  x += somefunction();\n } catch(e) {\n }\n document.write(x);\n}\n\na();\na();\n-----\n\nIt gets compiled to:\n\n-----\nfunction b() {\n  var a;\n  try {\n    a = &quot;1&quot; + somefunction()\n  }catch(c) {\n  }\n  try {\n    a = a + &quot;2&quot; + somefunction()\n  }catch(d) {\n  }\n  document.write(a)\n}\nb();\nb();\n-----\n\n<b>What is the expected output? What do you see instead?</b>\n\nThe problem is that it's including the constant &quot;1&quot; and &quot;2&quot; inside the try bloc

In [ ]:
summaries = []

for index, row in filtered_df.iterrows():
    print(f"Processing index {index}...")

    bug_id = row['bug.id']
    bug_report = row['bug_report']

    try:
        response = chain.invoke({"few_shot_examples": few_shot_examples,"bug_report": bug_report})
        print(response)
        summary_result = response.content.strip()
    except Exception as e:
        print(f"Error at index {index}: {e}")
        summary_result = "Summary not available"

    if not summary_result:
        summary_result = "Summary not available"

    print(f"Generated summary for index {index}: {summary_result}\n")
    summaries.append([bug_id, summary_result])

    time.sleep(0.1)


Processing index 65...
content='Function arguments should not be optimized away' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 2424, 'total_tokens': 2431, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C2ZLCuqVSE5iJpWFnIBgoNxpA6Ba3', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--59fd70bb-9ac5-4e5a-8d69-a24404698ad3-0' usage_metadata={'input_tokens': 2424, 'output_tokens': 7, 'total_tokens': 2431, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Generated summary for index 65: Function arguments should not be optimized away

Processing index 66...
content='Combine @interface and multiple @extends 

In [ ]:
summary_df = pd.DataFrame(summaries, columns=['Bug ID', 'Summary'])
summary_df.to_csv("/content/drive/MyDrive/gpt3.5_bug_summaries_few_shot.csv", index=False)

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=api_key, temperature=0.7, model="gpt-3.5-turbo")

In [ ]:
def chunk_code(text: str, max_chars: int = 9000):
    text = text if isinstance(text, str) else ""
    if not text.strip():
        return []
    return [text[i:i + max_chars] for i in range(0, len(text), max_chars)]

In [ ]:
chunk_template = PromptTemplate(
    input_variables=["code_chunk"],
    template=(
        "You are a senior software engineer helping to analyze this buggy code. Summarize this piece of buggy code in 1-2 sentences:\n"
        "Code Snippet:\n```{code_chunk}```\n\nSummary:"
    ),
)

In [ ]:
final_template = PromptTemplate(
    input_variables=["chunk_summaries_text"],
    template=(
        "Write a summary describing the main context of the bug using minimal words in strict 1 sentence.\n"
        "{chunk_summaries_text}\n"
        "Final Summary:"
    ),
)

In [ ]:
chunk_chain = chunk_template | llm
final_chain = final_template | llm

In [ ]:
final_summary_list = []
per_chunk_summary = []

In [ ]:
import time

In [ ]:
for index, row in filtered_df.iterrows():
    bug_id = row["bug.id"]
    buggy_code = str(row.get("buggy_code", "") or "")

    print(f"Processing bug.id={bug_id} ...")
    if not buggy_code.strip():
        final_summary = "No buggy code provided."
        chunk_summaries = []
    else:
        chunks = chunk_code(buggy_code, max_chars=9000)


        chunk_summaries = []
        for i, ch in enumerate(chunks, 1):
            try:
                s = chunk_chain.invoke({"code_chunk": ch})
            except Exception as e:
                s = f"Error summarizing chunk {i}: {e}"
            s_chunk=s.content.strip()
            print(s_chunk)
            chunk_summaries.append(s_chunk)
            time.sleep(0.15)

        joined = "\n".join(f"- {s}" for s in chunk_summaries) if chunk_summaries else "No chunk summaries."
        try:
            final_summary = final_chain.invoke({"chunk_summaries_text": joined})
            final_code_summary=final_summary.content.strip()
            print(f"final code summary: {final_code_summary}")
        except Exception as e:
            final_summary = f"Error {e}"


    final_summary_list.append([bug_id, final_code_summary])
    per_chunk_summary.append([ bug_id,joined])

Processing bug.id=66 ...
This piece of code is a part of a compiler pass called "RemoveUnusedVars" that performs garbage collection for variable and function definitions by traversing the JavaScript parse tree and marking referenced variables. It also handles assignments and continuations for variables that may be unreferenced.
This buggy code is analyzing JavaScript functions to remove unreferenced arguments and optimize call sites by modifying function signatures and call sites. It also tracks variables and scopes to determine which elements can be safely removed.
This piece of code is focused on analyzing and potentially removing references to parameters in function calls. It includes functions to check if a parameter can be removed from call sites, remove arguments from call sites, and interpret assignments to variables. The code aims to simplify and optimize function call signatures by removing unused or side-effect free parameters.
This piece of code contains methods to mark refe

In [ ]:
final_summary_list

[[66,
  'This code analyzes JavaScript functions to remove unused arguments, optimize call sites, and simplify function call signatures by tracking variables and scopes.'],
 [67,
  'This buggy code snippet contains methods for type checking properties, inheritance, and prototype modifications in JavaScript objects, with bugs related to property conflict checking in interfaces, validation of function call parameters, and handling of return statements.'],
 [68,
  'Code snippet analyzing variable inlining potential encounters bug due to incorrectly accessing parent node in inner loop when iterating through ancestors to check conditions for inlining.'],
 [69,
  'Code defines a `NamedType` class for named references to other types, resolving through a type registry or scope properties, with methods for type lookup, cycle handling, and property storage.'],
 [70,
  'Bug in code manipulating object literals in JavaScript, causing issues with variable declarations, assignments, and references.'

In [ ]:
per_chunk_summary

[[66,
  '- This piece of code is a part of a compiler pass called "RemoveUnusedVars" that performs garbage collection for variable and function definitions by traversing the JavaScript parse tree and marking referenced variables. It also handles assignments and continuations for variables that may be unreferenced.\n- This buggy code is analyzing JavaScript functions to remove unreferenced arguments and optimize call sites by modifying function signatures and call sites. It also tracks variables and scopes to determine which elements can be safely removed.\n- This piece of code is focused on analyzing and potentially removing references to parameters in function calls. It includes functions to check if a parameter can be removed from call sites, remove arguments from call sites, and interpret assignments to variables. The code aims to simplify and optimize function call signatures by removing unused or side-effect free parameters.\n- This piece of code contains methods to mark reference

In [ ]:
summary_df = pd.DataFrame(final_summary_list, columns=['Bug ID', 'Code_Summary'])
summary_df.to_csv("/content/drive/MyDrive/gpt3.5_full_code_summary.csv", index=False)

In [ ]:
summary_df1 = pd.DataFrame(per_chunk_summary, columns=['Bug ID', 'Chunk_Code_Summary'])
summary_df1.to_csv("/content/drive/MyDrive/gpt3.5_chunk_code_summary.csv", index=False)

In [ ]:
output_file = "/content/drive/My Drive/active-bugs.csv"
gt_file = "/content/drive/My Drive/gt-summaries.csv"
code_summ="/content/drive/MyDrive/gpt3.5_full_code_summary.csv"

df = pd.read_csv(output_file)


if 'example_summary ' in df.columns:
    df = df.drop(columns=['example_summary '])

gt = pd.read_csv(gt_file)
df = df.merge(gt, on='bug.id', how='left')
cs=pd.read_csv(code_summ)


In [ ]:
cs=cs.rename(columns={"Bug ID": "bug.id"})

In [ ]:
df = df.merge(cs, on='bug.id', how='left')

In [ ]:
filtered_df = df[
    (df['project.id'] == 3) &
    (df['report.url'].notna()) & (df['report.url'] != '') &
    (df['buggy.url'].notna()) & (df['buggy.url'] != '') &
    (df['fixed.url'].notna()) & (df['fixed.url'] != '') &
    (df['diff.url'].notna()) & (df['diff.url'] != '') &
    (df['bug_report'].notna()) & (df['bug_report'] != '')
]


In [ ]:
filtered_df.head()

,bug.id,project.name,project.id,revision.id.buggy,revision.id.fixed,report.id,report.url,buggy.url,fixed.url,diff.url,bug_report,buggy_code,patch_code,ground_truth_summary,Code_Summary
65,66,Closure,3,2353d807058bc2a20af279a480d6652cdf892f4d,1dfad5043a207e032a78ef50c3cba50488bcd300,253,https://storage.googleapis.com/google-code-arc...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,Bug Report ID: 253\nStatus: Fixed\nSummary: fu...,/*\n * Copyright 2008 The Closure Compiler Aut...,Commit Message: fixed files form Closure#1\nFi...,Summary: function arguments should not be opti...,This code analyzes JavaScript functions to rem...
66,67,Closure,3,61095090415cff7cae4f3645fa76ee7cdd3ee23d,d1cfe67977d8f3aaa85ec20c262171da394d5977,884,https://storage.googleapis.com/google-code-arc...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,Bug Report ID: 884\nStatus: Fixed\nSummary: co...,/*\n * Copyright 2006 The Closure Compiler Aut...,Commit Message: fixed files form Closure#2\nFi...,combining @interface and multiple @extends can...,This buggy code snippet contains methods for t...
67,68,Closure,3,3cc85c3c37aa8bc834a4a86f91ddeb399d854024,d80fcc04239ab8c4cf781273c4f9bc54cf06f479,864,https://storage.googleapis.com/google-code-arc...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,Bug Report ID: 864\nStatus: Fixed\nSummary: op...,/*\n * Copyright 2009 The Closure Compiler Aut...,Commit Message: fixed files form Closure#3\nFi...,optimization fails with variable in catch clause,Code snippet analyzing variable inlining poten...
68,69,Closure,3,1c95684b4a6add525b3070cbd27c234981520676,efefb736fccc2039b5fb079710b3f2ac82b8c6e4,873,https://storage.googleapis.com/google-code-arc...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,Bug Report ID: 873\nStatus: Fixed\nSummary: Co...,/*\n *\n * ***** BEGIN LICENSE BLOCK *****\n *...,Commit Message: fixed files form Closure#4\nFi...,Converting from an interface type to a constru...,Code defines a `NamedType` class for named ref...
69,70,Closure,3,722d1192e7ed174a12911dce09594228e31240e9,59eec92e364b2ec2cec9dd63449f5c0134983f18,851,https://storage.googleapis.com/google-code-arc...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,https://github.com/program-repair/defects4j-di...,Bug Report ID: 851\nStatus: Fixed\nSummary: Co...,/*\n * Copyright 2011 The Closure Compiler Aut...,Commit Message: fixed files form Closure#5\nFi...,"Compiler ignores 'delete' statements, can brea...",Bug in code manipulating object literals in Ja...


In [ ]:
from langchain import PromptTemplate
from langchain_openai import OpenAI
from langchain.chains import LLMChain


In [ ]:
from langchain_openai import ChatOpenAI

llm = OpenAI(api_key=api_key, temperature=0.7)



In [ ]:
template = """
Given the bug report with buggy code summary, Write a one-sentence summary of the core issue using no more than 10 words.\n

Bug Report:
{bug_report}

Buggy Code Summary:
{Code_Summary}

Summary:
"""

prompt = PromptTemplate(input_variables=["bug_report","Code_Summary"], template=template)
chain = prompt | llm


In [ ]:
import time

In [ ]:
summaries = []

for index, row in filtered_df.iterrows():
    print(f"Processing index {index}...")

    bug_report = row['bug_report']
    Code_Summary=row['Code_Summary']
    bug_id = row['bug.id']

    try:
        result = chain.invoke({"bug_report": bug_report,
                               "Code_Summary": Code_Summary})
        print(result)
        summary = result.strip()
    except Exception as e:
        print(f"Error on index {index}: {e}")
        summary = "Summary not available"

    print(f"Generated summary: {summary}\n")
    summaries.append([bug_id, summary])

    time.sleep(0.1)


Processing index 65...
Function arguments are being optimized away, compromising the function's length property.
Generated summary: Function arguments are being optimized away, compromising the function's length property.

Processing index 66...
Compiler crashes when combining @interface and multiple @extends.
Generated summary: Compiler crashes when combining @interface and multiple @extends.

Processing index 67...
Variable inlining bug caused by incorrect parent node access.
Generated summary: Variable inlining bug caused by incorrect parent node access.

Processing index 68...
Conversion from interface type to constructor causes stack overflow.
Generated summary: Conversion from interface type to constructor causes stack overflow.

Processing index 69...
Compiler ignores 'delete' statements on rewritten object variables, causing issues.
Generated summary: Compiler ignores 'delete' statements on rewritten object variables, causing issues.

Processing index 70...
Type checking fails 

In [ ]:
summaries

[[66,
  "Function arguments are being optimized away, compromising the function's length property."],
 [67, 'Compiler crashes when combining @interface and multiple @extends.'],
 [68, 'Variable inlining bug caused by incorrect parent node access.'],
 [69, 'Conversion from interface type to constructor causes stack overflow.'],
 [70,
  "Compiler ignores 'delete' statements on rewritten object variables, causing issues."],
 [71, "Type checking fails to detect mismatched 'this' types."],
 [72, 'Buggy type inference in goog.isFunction and related methods.'],
 [73,
  'Obfuscated code triggers TypeError in Firefox due to CollapseVariableDeclarations class.'],
 [74, 'Buggy CommonJS module rewriting causes errors in compilation.'],
 [75, 'Wrong code generated when mixing types in ternary operator.'],
 [76,
  'Type checking JavaScript expressions with internal errors and warnings.'],
 [77, 'Missing null checks in data flow analysis causes buggy behavior.'],
 [78,
  'Peephole optimizations class

In [ ]:
summary_df = pd.DataFrame(summaries, columns=['Bug ID', 'Summary'])
summary_df.to_csv("/content/drive/MyDrive/gpt3.5_bug_summaries_with_code_zero_shot.csv", index=False)


In [ ]:
example_bug_report = filtered_df.iloc[5]['bug_report']
example_summary = filtered_df.iloc[5]['ground_truth_summary']
example_code_summary=filtered_df.iloc[5]['Code_Summary']

In [ ]:
template = """Here is an example of a bug report, summarized buggy code and its summary:

Example Bug Report:
{example_bug_report}

Example Buggy Code Summary:
{example_code_summary}

Example Summary:
{example_summary}

Now, Given a Bug Report with Buggy Code Summary, write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless it naturally applies; tailor the summary to the new bug report.

Bug Report:
{bug_report}

Buggy Code Summary:
{Code_Summary}

Summary:"""

prompt_template = PromptTemplate(
    input_variables=["example_bug_report", "example_code_summary", "example_summary", "bug_report","Code_Summary"],
    template=template
)
chain = prompt_template | llm

In [ ]:
import time

In [ ]:
summaries = []

for index, row in filtered_df.iterrows():
    print(f"Processing index {index}...")

    bug_report = row['bug_report']
    bug_id = row['bug.id']
    Code_Summary=row['Code_Summary']

    try:
        result = chain.invoke({
            "example_bug_report": example_bug_report,
            "example_code_summary": example_code_summary,
            "example_summary": example_summary,
            "bug_report": bug_report,
            "Code_Summary": Code_Summary
        })
        print(result)
        summary = result.strip()
    except Exception as e:
        print(f"Error on index {index}: {e}")
        summary = "Summary not available"

    print(f"Generated summary: {summary}\n")
    summaries.append([bug_id, summary])

    time.sleep(0.1)


Processing index 65...
 function arguments are being optimized away, compromising code
Generated summary: function arguments are being optimized away, compromising code

Processing index 66...

Combining @interface and multiple @extends crashes compiler.
Generated summary: Combining @interface and multiple @extends crashes compiler.

Processing index 67...

Optimization fails due to incorrect variable access in inlining loop.
Generated summary: Optimization fails due to incorrect variable access in inlining loop.

Processing index 68...
 Infinite recursion in PrototypeObjectType.isSubtype due to cycles.
Generated summary: Infinite recursion in PrototypeObjectType.isSubtype due to cycles.

Processing index 69...
 Compiler ignores 'delete' statements, leading to unexpected behavior in object manipulation.
Generated summary: Compiler ignores 'delete' statements, leading to unexpected behavior in object manipulation.

Processing index 70...
 Type mismatch warnings for JavaScript type annot

In [ ]:
summary_df = pd.DataFrame(summaries, columns=['Bug ID', 'Summary'])
summary_df.to_csv("/content/drive/MyDrive/gpt3.5_bug_summaries_with_code_one_shot.csv", index=False)

In [ ]:
examples = []
example_indices = [11, 21, 31]

for i in example_indices:
    ex_bug = filtered_df.iloc[i]['bug_report']
    ex_code_sum=filtered_df.iloc[i]['Code_Summary']
    ex_sum = filtered_df.iloc[i]['ground_truth_summary']
    examples.append((ex_bug,ex_code_sum, ex_sum))

In [ ]:
few_shot_examples = ""
for i, (bug,codeSum, summary) in enumerate(examples, 1):
    few_shot_examples += f"""Example {i} Bug Report:
{bug}

Example {i} Buggy Code Summary:
{codeSum}

Example {i} Summary:
{summary}

"""
template = """
Here are a few examples of bug reports,buggy code summaries and their summaries:

{few_shot_examples}
Now, Given a Bug Report with Buggy Code Summary, write a one-sentence summary of the core issue using no more than 10 words.Tailor the summary to the new bug report. Avoid copying earlier examples unless they naturally apply.

Bug Report:
{bug_report}

Buggy Code Summary:
{Code_Summary}

Summary:
"""
prompt_template = PromptTemplate(
    input_variables=["few_shot_examples", "bug_report","Code_Summary"],
    template=template
)

chain = prompt_template | llm

In [ ]:
few_shot_examples

"Example 1 Bug Report:\nBug Report ID: 794\nStatus: Fixed\nSummary: Try/catch blocks incorporate code not inside original blocks\nLabels: Type-Defect, Priority-High\nStars: 0\nComment Count: 3\nComments:\n\n0. **Comment by User (ID: 8682039881586643116)**\n   - **Timestamp**: 1344189644\n   - **Content**: <b>What steps will reproduce the problem?</b>\n\nStarting with this code:\n\n-----\nfunction a() {\n var x = '1';\n try {\n  x += somefunction();\n } catch(e) {\n }\n x += &quot;2&quot;;\n try {\n  x += somefunction();\n } catch(e) {\n }\n document.write(x);\n}\n\na();\na();\n-----\n\nIt gets compiled to:\n\n-----\nfunction b() {\n  var a;\n  try {\n    a = &quot;1&quot; + somefunction()\n  }catch(c) {\n  }\n  try {\n    a = a + &quot;2&quot; + somefunction()\n  }catch(d) {\n  }\n  document.write(a)\n}\nb();\nb();\n-----\n\n<b>What is the expected output? What do you see instead?</b>\n\nThe problem is that it's including the constant &quot;1&quot; and &quot;2&quot; inside the try bloc

In [ ]:
import time

In [ ]:
summaries = []

for index, row in filtered_df.iterrows():
    print(f"Processing index {index}...")

    bug_id = row['bug.id']
    bug_report = row['bug_report']
    Code_Summary=row['Code_Summary']

    try:
        response = chain.invoke({"few_shot_examples": few_shot_examples,"bug_report": bug_report,"Code_Summary": Code_Summary})
        print(response)
        summary_result = response.strip()
    except Exception as e:
        print(f"Error at index {index}: {e}")
        summary_result = "Summary not available"

    if not summary_result:
        summary_result = "Summary not available"

    print(f"Generated summary for index {index}: {summary_result}\n")
    summaries.append([bug_id, summary_result])

    time.sleep(0.01)


Processing index 65...
Function arguments should not be optimized away
Generated summary for index 65: Function arguments should not be optimized away

Processing index 66...
Combining @interface with multiple @extends crashes compiler
Generated summary for index 66: Combining @interface with multiple @extends crashes compiler

Processing index 67...
Optimization fails with variable in catch clause.
Generated summary for index 67: Optimization fails with variable in catch clause.

Processing index 68...
Converting from interface to constructor causes stack overflow.
Generated summary for index 68: Converting from interface to constructor causes stack overflow.

Processing index 69...
Compiler ignores 'delete' statements, causing unexpected behavior.
Generated summary for index 69: Compiler ignores 'delete' statements, causing unexpected behavior.

Processing index 70...
Better 'this' type checking
Generated summary for index 70: Better 'this' type checking

Processing index 71...
Bad t

In [ ]:
summary_df = pd.DataFrame(summaries, columns=['Bug ID', 'Summary'])
summary_df.to_csv("/content/drive/MyDrive/gpt3.5_bug_summaries_with_code_few_shot.csv", index=False)